#### Load packages

In [19]:
from google import genai

In [20]:
from google.genai import types

In [21]:
import os
import json
import time
import re
import pandas as pd

#### Add Gemini API key and client

In [22]:
GEMINI_API_KEY = ''

In [23]:
client = genai.Client(api_key=GEMINI_API_KEY)

#### Function for prompt for origin in Theme

In [25]:
def build_originInTheme_prompt(story):
    prompt = f"""
    You are a creative writing expert.
    Read the following short story:
    
    {story}
    
    Evaluate the **originality of theme and content** in the story on a scale from 0 to 3, where:
    
    - 0 = Common or overused themes; offers little new insight.
    - 1 = Slightly fresh perspective or small thematic twist on familiar ideas.
    - 2 = Distinctive and thought-provoking themes; provides meaningful new insight.
    - 3 = Highly original themes or content; offers profound, unique, or transformative ideas.
    
    Use the following background as guidance:
    
    Originality in theme and content refers to the extent to which a work offers new ideas, perspectives, or thematic territory.  
    A story with high originality may:
    - Provide unique insights into human nature, society, or culture.
    - Explore concepts rarely addressed in literature.
    - Reframe familiar themes in surprising or revelatory ways.
    - Present bold philosophical, speculative, or moral ideas that stimulate reflection.
    
    For example:
    - *Never Let Me Go* by Kazuo Ishiguro uses a quiet, intimate narrative to explore the ethics of cloning and mortality.
    - *Kindred* by Octavia E. Butler combines time travel with slavery narratives to reframe historical trauma.
    - *The Left Hand of Darkness* by Ursula K. Le Guin explores gender and identity through an alien society with no fixed sex.
     
    Original themes often linger in the reader’s mind, sparking conversation and inspiring new ways of thinking.
    
    Respond in the format:
    
    Reasoning: [your analysis]  
    Score: [0–3]  
    """

    return prompt

#### Load stories

In [26]:
file_path = 'NewYorkerStories.json'

with open(file_path, 'r') as f:
    stories = json.load(f)

#### Send stories to Gemini and save response

In [29]:
# Set up directory
output_dir = "instruction_tests/Gemini/OriginTheme/prompt_01"
os.makedirs(output_dir, exist_ok=True)

# Initialise dataframe for results
scores = pd.DataFrame()

# Loop through each story
for story in stories[0:]:
    thoughts = ""
    answer = ""

    print('---------Story:',story['story_id'],'--------')

    # Build prompt
    prompt = build_oringinInTheme_prompt(story['content'])

    # Send prompt to Gemini
    response = client.models.generate_content(
        model="gemini-2.5-flash", 
        contents=prompt,
        config=types.GenerateContentConfig(
            thinking_config=types.ThinkingConfig(
              include_thoughts=True
            )
        )
    )

    # Record the number of tokens used to ensure not overusing
    total_tokens = response.usage_metadata.total_token_count
    print("Total tokens used: ", total_tokens)

    # Print the 'thoughts' of the model and the response
    if response.candidates:
        for part in response.candidates[0].content.parts:
          if not part.text:
            continue
          if part.thought:
            print("Thought summary:")
            print(part.text)
            thoughts = part.text
            print()
          else:
            print("Answer:")
            print(part.text)
            answer = part.text
            print()

        # Get score out of answer
        match = re.search(r'Score:\s*(\d+(?:\.\d+)?)', answer)
        if match:
            score = float(match.group(1))
        else:
            score = "#N/A"
    else:
        print("Didn't return content")
        print("Block Reason:", response.prompt_feedback.block_reason)
        score = "#N/A"
        thoughts = 'N/A'
        answer = 'N/A'
        
    # Save response
    new_row = pd.DataFrame([{"story": story['story_id'].split("_",1)[0], "writer": story['story_id'].split("_",1)[1], "score":score}])

    filename = os.path.join(output_dir,'scores.csv')
    
    if not os.path.exists(filename):
        new_row.to_csv(filename, mode='w', index=False)
    else:
        new_row.to_csv(filename, mode='a', index=False, header=False)

    # Save response and score
    output_path = os.path.join(output_dir, f"{story['story_id']}.json")
    with open(output_path, "w") as f:
        json.dump({"story_name": story['story_name'],'story_writer':story['story_id'], 'thought':thoughts, 'answer':answer, 'score': score}, f, indent=2)

    #Sleep so as to not overwhelm the model
    time.sleep(15)

---------Story: 0_NewYorker --------
Total tokens used:  5382
Thought summary:
**Assessing the Story's Originality**

As an expert, I've approached this task with a keen eye for novelty and depth. I recognize the common tropes: the hospital setting, the physical struggles of postpartum recovery. These are familiar, even overused, territory. However, it's the *manner* in which the story engages these tropes that warrants close examination.

The narrative dives headfirst into the surreal, and that's where its originality blossoms. While the underlying anxieties of new motherhood are increasingly explored in contemporary literature, this story doesn't just depict them; it *embodies* them through potent, unsettling imagery. The "wrong child," the shared delusion among mothers, the bizarre whitewashing ritual - these are not merely descriptive elements; they are symbolic portals to a profoundly distorted reality.

The core of its originality lies in how it externalizes these internal strugg